In [24]:
import nltk
nltk.download('brown')
import sys
from nltk.corpus import brown
brown_tags_words = [ ]
for sent in brown.tagged_sents():
    # 先加开头
    brown_tags_words.append( ("START", "START") )
    # 为了省事儿，我们把tag都省略成前两个字母
    brown_tags_words.extend([ (tag[:2], word) for (word, tag) in sent ])
    # 加个结尾
    brown_tags_words.append( ("END", "END") )

[nltk_data] Downloading package brown to /Users/hcnucai/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [25]:
#词统计
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words);
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords,nltk.MLEProbDist);
#查看统计结果
print ("The probability of an adjective (JJ) being 'new' is", cpd_tagwords["JJ"].prob("new"));
print("The probability of a verb (VB) being 'duck' is", cpd_tagwords["VB"].prob("duck"))


The probability of an adjective (JJ) being 'new' is 0.01472344917632025
The probability of a verb (VB) being 'duck' is 6.042713350943527e-05


In [26]:
brown_tags = [tag for (tag, word) in brown_tags_words ]
# count(t{i-1} ti)
# bigram的意思是 前后两个一组，联在一起
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
# P(ti | t{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)
print("If we have just seen 'DT', the probability of 'NN' is", cpd_tags["DT"].prob("NN"))
print( "If we have just seen 'VB', the probability of 'JJ' is", cpd_tags["VB"].prob("DT"))
print( "If we have just seen 'VB', the probability of 'NN' is", cpd_tags["VB"].prob("NN"))

If we have just seen 'DT', the probability of 'NN' is 0.5057722522030194
If we have just seen 'VB', the probability of 'JJ' is 0.016885067592065053
If we have just seen 'VB', the probability of 'NN' is 0.10970977711020183


In [27]:
prob_tagsequence = cpd_tags["START"].prob("PP") * cpd_tagwords["PP"].prob("I") * \
    cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("want") * \
    cpd_tags["VB"].prob("TO") * cpd_tagwords["TO"].prob("to") * \
    cpd_tags["TO"].prob("VB") * cpd_tagwords["VB"].prob("race") * \
    cpd_tags["VB"].prob("END")

print( "The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is:", prob_tagsequence)

The probability of the tag sequence 'START PP VB TO VB END' for 'I want to race' is: 1.0817766461150474e-14


In [28]:
#viterbi的实现
distinct_tags = set(brown_tags);
sentence = ["I","want","to","race"];
senlen = len(sentence);
#开始维特比

In [29]:
viterbi = [];
backpointer = [];
first_viterbi = {};
first_backpointer = {};
for tag in distinct_tags:
    if tag == 'START':
        continue;
    first_viterbi[ tag ] = cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
    first_backpointer[ tag ] = "START"

print(first_viterbi)
print(first_backpointer)

{':-': 0.0, '``': 0.0, '.': 0.0, 'WD': 0.0, 'NN': 1.0580313619573935e-06, 'AP': 0.0, '(-': 0.0, ')': 0.0, "''": 0.0, 'END': 0.0, ':': 0.0, 'PN': 0.0, '*-': 0.0, 'PP': 0.014930900689060006, 'FW': 0.0, 'DT': 0.0, ')-': 0.0, 'WQ': 0.0, 'IN': 0.0, 'AT': 0.0, 'NP': 1.7319067623793952e-06, 'VB': 0.0, 'OD': 0.0, 'HV': 0.0, '(': 0.0, 'RB': 0.0, ',-': 0.0, 'RP': 0.0, ',': 0.0, 'QL': 0.0, 'BE': 0.0, 'CS': 0.0, '--': 0.0, 'CD': 0.0, 'UH': 0.0, 'CC': 0.0, 'NR': 0.0, 'TO': 0.0, 'DO': 0.0, 'AB': 0.0, 'JJ': 0.0, 'MD': 0.0, 'WP': 0.0, "'": 0.0, 'NI': 3.3324520848931064e-07, '*': 0.0, 'WR': 0.0, '.-': 0.0, 'EX': 0.0, 'RN': 0.0}
{':-': 'START', '``': 'START', '.': 'START', 'WD': 'START', 'NN': 'START', 'AP': 'START', '(-': 'START', ')': 'START', "''": 'START', 'END': 'START', ':': 'START', 'PN': 'START', '*-': 'START', 'PP': 'START', 'FW': 'START', 'DT': 'START', ')-': 'START', 'WQ': 'START', 'IN': 'START', 'AT': 'START', 'NP': 'START', 'VB': 'START', 'OD': 'START', 'HV': 'START', '(': 'START', 'RB': 'S

In [30]:
viterbi.append(first_viterbi);
backpointer.append(first_backpointer);
currbest = max(first_viterbi.keys(), key = lambda tag: first_viterbi[ tag ])
print( "Word", "'" + sentence[0] + "'", "current best two-tag sequence:", first_backpointer[ currbest], currbest)

Word 'I' current best two-tag sequence: START PP


In [31]:
#开始循环
for wordindex in range(1, len(sentence)):
    this_viterbi = { }
    this_backpointer = { }
    prev_viterbi = viterbi[-1]
    
    for tag in distinct_tags:
        # START没有卵用的，我们要忽略
        if tag == "START": continue
        
        # 如果现在这个tag是X，现在的单词是w，
        # 我们想找前一个tag Y，并且让最好的tag sequence以Y X结尾。
        # 也就是说
        # Y要能最大化：
        # prev_viterbi[ Y ] * P(X | Y) * P( w | X)
        
        best_previous = max(prev_viterbi.keys(),
                            key = lambda prevtag: \
            prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob(tag) * cpd_tagwords[tag].prob(sentence[wordindex]))

        this_viterbi[ tag ] = prev_viterbi[ best_previous] * \
            cpd_tags[ best_previous ].prob(tag) * cpd_tagwords[ tag].prob(sentence[wordindex])
        this_backpointer[ tag ] = best_previous
    
    # 每次找完Y 我们把目前最好的 存一下
    currbest = max(this_viterbi.keys(), key = lambda tag: this_viterbi[ tag ])
    print( "Word", "'" + sentence[ wordindex] + "'", "current best two-tag sequence:", this_backpointer[ currbest], currbest)


    # 完结
    # 全部存下来
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)


# 找所有以END结尾的tag sequence
prev_viterbi = viterbi[-1]
best_previous = max(prev_viterbi.keys(),
                    key = lambda prevtag: prev_viterbi[ prevtag ] * cpd_tags[prevtag].prob("END"))

prob_tagsequence = prev_viterbi[ best_previous ] * cpd_tags[ best_previous].prob("END")

# 我们这会儿是倒着存的。。。。因为。。好的在后面
best_tagsequence = [ "END", best_previous ]
# 同理 这里也有倒过来
backpointer.reverse()

Word 'want' current best two-tag sequence: PP VB
Word 'to' current best two-tag sequence: VB TO
Word 'race' current best two-tag sequence: IN NN


In [32]:
current_best_tag = best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]
best_tagsequence.reverse()
print( "The sentence was:", end = " ")
for w in sentence: print( w, end = " ")
print("\n")
print( "The best tag sequence is:", end = " ")
for t in best_tagsequence: print (t, end = " ")
print("\n")
print( "The probability of the best tag sequence is:", prob_tagsequence)

The sentence was: I want to race 

The best tag sequence is: START PP VB IN NN END 

The probability of the best tag sequence is: 5.71772824864617e-14
